In [11]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

In [5]:
def sistema_dinamico(t, y, u, params):
    Sh, Ih, Rh, Sv, Iv = y
    Lambda_h, omega, mu_h, beta_h, xi1, q1, delta, rho, Lambda_v, mu_v, beta_v, xi2, q2 = params
    
    Nh = Sh + Ih + Rh
    Nv = Sv + Iv

    # Convertir 'u' a escalar si es necesario
    if np.isscalar(u) == False:
        u = u.item()  # Convierte 'u' a escalar si es un array de un elemento

    # Ecuaciones del sistema incluyendo el término de control 'u'
    dSh_dt = Lambda_h + omega * Rh - beta_h * Iv / Nh * Sh - mu_h * Sh
    dIh_dt = beta_h * Iv / Nh * Sh - (xi1 * (1 - q1) + u) * Ih - (delta + rho + mu_h) * Ih
    dRh_dt = (xi1 * (1 - q1) + u) * Ih + delta * Ih - (omega + mu_h) * Rh
    dSv_dt = Lambda_v - beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Sv - mu_v * Sv
    dIv_dt = beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Iv - mu_v * Iv

    return [dSh_dt, dIh_dt, dRh_dt, dSv_dt, dIv_dt]
def control_optimo(u, *args):
    T, initial_conditions, params = args
    result = solve_ivp(lambda t, y: sistema_dinamico(t, y, u, params), [0, T], initial_conditions, t_eval=np.linspace(0, T, 100))
    Ih = result.y[1]  # Infectados humanos
    coste = np.trapz(Ih**2 + u**2, result.t)  # Función de coste
    return coste
# Parámetros del modelo y condiciones iniciales
params = (1, 0.1, 0.01, 0.4, 0.7, 0.1, 0.05, 0.02, 0.5, 0.1, 0.3, 0.3, 0.05)
initial_conditions = [1000, 100, 50, 1000, 100]
T = 50  # Horizonte de tiempo



In [28]:
# Parámetros del modelo y condiciones iniciales
params = (1, 0.1, 0.01, 0.4, 0.7, 0.1, 0.05, 0.02, 0.5, 0.1, 0.3, 0.3, 0.05)
initial_conditions = [1000, 100, 50, 1000, 100]
T = 50  # Horizonte de tiempo
C= 1 #Coste de infectados, arbitrario

#Condiciones iniciales
Sh0 = 1000      # Initial susceptible humans
Ih0 = 100       # Initial infected humans
Rh0 = 50        # Initial recovered humans
Sv0 = 1000      # Initial susceptible mosquitoes
Iv0 = 100       # Initial infected mosquitoes
#The initial conditons for the adjoints, even if they should have a more important meaning for now they will be 0
lambda_1_0=0
lambda_2_0=0
lambda_3_0=0
lambda_4_0=0
lambda_5_0=0
#Faltan condiciones iniciales para el control. mas investigacion seria sugerida, basado en que es la efectividad
#que inicie en 1 deberia ser una aproximación adecuada
theta_1_0=1
theta_2_0=1

Lambda_h, omega, mu_h, beta_h, xi1, q1, delta, rho, Lambda_v, mu_v, beta_v, xi2, q2 = params
epsilon = 1
#It is noted epsilon doesn´t appear and a capital bitting rate of 1 would be weird. Data on epsilon is not
#readily available on the paper yet on the sensitivility analisis it is. Intially we will leave it as it is
#theta_1 = 1
#theta_2 = 1
#Other missing parameters, the recovery rate due to the drug, 1 is fine for tests-era en control-
C= 1 #Coste de infectados, arbitrario
d1=1 #Coste de drogas humanas, arbitrario
d2=1 #Coste de insecticidas, arbitrario
    

#Derivamos las variables adjuntas
#La primera parte deberia ser similar. El control u no es necesario y el tiempo no creo pero por si acaso se deja.
def Sistema(t, y):
    Sh, Ih, Rh, Sv, Iv,lambda_1,lambda_2,lambda_3,lambda_4,lambda_5 = y
   
    Nh = Sh + Ih + Rh
    Nv = Sv + Iv
    #Variables adjuntas
    # In the first the Iv therm is coppied twice, I think it should be Sh
    Dlambda_1_Dt = - ((Sh * beta_h * epsilon * Iv * (Ih - Rh)(lambda_2 - lambda_1)) / (Nh*Nh)) + lambda_1 * mu_h + (Sv * beta_v * epsilon * Ih(lambda_5 - lambda_4) / (Nh * Nh))
    Dlambda_2_Dt = - C - xi1 * theta_1 * (1-q1) * (lambda_3 - lambda_2) + lambda_2 * mu_h - delta * (lambda_3 - lambda_2) + Sh*beta_h*epsilon*Iv/(Nh^2) - (Sv*beta_v*epsilon*(Sh+Rh)*(lambda_5-lambda_4)/(Nh^2))#sospecho que el último termino es realmente positivo o el penultmo negativo
    Dlambda_3_Dt =  - omega*lambda_1 + mu_h*lambda_3 + (Sh*beta_h*epsilon*Iv*(lambda_2-lambda_1)/(Nh^2)) + (Sv*beta_v*epsilon*Ih*(lambda_5-lambda_4)/(Nh^2))
    Dlambda_4_Dt = - (lambda_4*beta_v*epsilon*Ih/Nh) + lambda_4*xi2*theta_2*(1-q2) + lambda_4*mu_v
    Dlambda_5_Dt = - C - (Sh*beta_h*epsilon(lambda_2-lambda_1)/Nh) + lambda_5*xi2*theta_2*(1-q2)+lambda_5 * mu_v
    #Sistema base
     # Ecuaciones del sistema incluyendo el término de control 'u'
    dSh_dt = Lambda_h + omega * Rh - beta_h * Iv / Nh * Sh - mu_h * Sh
    dIh_dt = beta_h * Iv / Nh * Sh - (xi1 * (1 - q1) + u) * Ih - (delta + rho + mu_h) * Ih
    dRh_dt = (xi1 * (1 - q1) + u) * Ih + delta * Ih - (omega + mu_h) * Rh
    dSv_dt = Lambda_v - beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Sv - mu_v * Sv
    dIv_dt = beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Iv - mu_v * Iv
    #control optimo segun el principio de Pontryagin
    Dtheta_1_Dt=xi1*(1-q1)*Ih*(lambda_2-lambda_3)/d1
    Dtheta_2_Dt=xi2*(1-q2)*(lambda_4*Sv-lambda_5*Iv)/d2
    
    
    return [Dlambda_1_Dt,Dlambda_2_Dt,Dlambda_3_Dt,Dlambda_4_Dt,Dlambda_5_Dt,dSh_dt,dIh_dt,dRh_dt,dSv_dt,dIh_dt,Dtheta_1_Dt,Dtheta_2_Dt]
    
    #Now to try to solve this the code from before ought to work
 # Time vector
t = np.linspace(0, 200, 400)

# Solve ODE
sol = solve_ivp(Sistema, [t.min(), t.max()], [Sh0, Ih0, Rh0, Sv0, Iv0,lambda_1_0,lambda_2_0,lambda_3_0,lambda_4_0,lambda_5_0], t_eval=t)


TypeError: 'numpy.float64' object is not callable

In [ ]:
#Basically the previous code but cleaner

#Se crea la función
#D carga con los parametros constantes, P con las variables, t significa tiempo
def SolvedSystem(D,P,t):
    
    #Le doy sus valores a los parametros; son 17,en orden:
    # 1/ Recrutamiento de personas, 2/ Ratio de perdida de inmunidad natural de humanos, 3/ Tasa de muerte natural de humanos,
    # 4/ Probabilidad de trasmisión mosquito-a-humano, 5/ Eficacia de la droga, 6/ Ratio de adquisición de resistencia,
    # 7/ Ratio de recuperación espontanea, 8/Muerte por infección. 9/ Recrutamiento de mosquitos,
    # 10/ Muerte natural de mosquitos, 11/ Probabilidad de infección humano-a-mosquito,12/ Eficacia de insecticida,
    # 13/ Resistencia a los insecticidas, 14/ Tasa de mordeduras per capita, 15/ Costo por Infectados,
    # 16/ Costo de drogas, 17/ Costo de insecticidas
    Lambda_h, omega, mu_h, beta_h, xi1, q1, delta, rho, Lambda_v, mu_v, beta_v, xi2, q2,epsilon,C,d1,d2 = D
    
    #Aclaro los nombres de las variables
    Sh, Ih, Rh, Sv, Iv,lambda_1,lambda_2,lambda_3,lambda_4,lambda_5,theta_1,theta_2 = P
    Nh = Sh + Ih + Rh
    Nv = Sv + Iv
    
    #Defino las ecuaciones de las variables originales
    DSh_dt = Lambda_h + omega * Rh - beta_h * Iv / Nh * Sh - mu_h * Sh
    DIh_dt = beta_h * Iv / Nh * Sh - (xi1 * (1 - q1) + u) * Ih - (delta + rho + mu_h) * Ih
    DRh_dt = (xi1 * (1 - q1) + u) * Ih + delta * Ih - (omega + mu_h) * Rh
    DSv_dt = Lambda_v - beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Sv - mu_v * Sv
    DIv_dt = beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Iv - mu_v * Iv
    #Las ecuaciones que gobiernan las variables de holgura
     Dlambda_1_Dt = - ((Sh * beta_h * epsilon * Iv * (Ih - Rh)(lambda_2 - lambda_1)) / (Nh*Nh)) + lambda_1 * mu_h + (Sv * beta_v * epsilon * Ih(lambda_5 - lambda_4) / (Nh * Nh))
    Dlambda_2_Dt = - C - xi1 * theta_1 * (1-q1) * (lambda_3 - lambda_2) + lambda_2 * mu_h - delta * (lambda_3 - lambda_2) + Sh*beta_h*epsilon*Iv/(Nh^2) - (Sv*beta_v*epsilon*(Sh+Rh)*(lambda_5-lambda_4)/(Nh^2))#sospecho que el último termino es realmente positivo o el penultmo negativo
    Dlambda_3_Dt =  - omega*lambda_1 + mu_h*lambda_3 + (Sh*beta_h*epsilon*Iv*(lambda_2-lambda_1)/(Nh^2)) + (Sv*beta_v*epsilon*Ih*(lambda_5-lambda_4)/(Nh^2))
    Dlambda_4_Dt = - (lambda_4*beta_v*epsilon*Ih/Nh) + lambda_4*xi2*theta_2*(1-q2) + lambda_4*mu_v
    Dlambda_5_Dt = - C - (Sh*beta_h*epsilon(lambda_2-lambda_1)/Nh) + lambda_5*xi2*theta_2*(1-q2)+lambda_5 * mu_v
    #Como deberia ser los controles según el principio de Pontryagin
    Dtheta_1_Dt=xi1*(1-q1)*Ih*(lambda_2-lambda_3)/d1
    Dtheta_2_Dt=xi2*(1-q2)*(lambda_4*Sv-lambda_5*Iv)/d2
    
    #La solución
    S=[Dlambda_1_Dt,Dlambda_2_Dt,Dlambda_3_Dt,Dlambda_4_Dt,Dlambda_5_Dt,dSh_dt,dIh_dt,dRh_dt,dSv_dt,dIh_dt,Dtheta_1_Dt,Dtheta_2_Dt]
    return S


#En este punto creo los parametros iniciales, se basan en los usados en el paper para areas rurales
Sh_0= 100000
Ih_0= 30000
Rh_0= 20000
Sv_0= 50000
Iv_0= 10000


In [ ]:
# Parámetros del modelo y condiciones iniciales
params = (1, 0.1, 0.01, 0.4, 0.7, 0.1, 0.05, 0.02, 0.5, 0.1, 0.3, 0.3, 0.05)
initial_conditions = [1000, 100, 50, 1000, 100]
T = 50  # Horizonte de tiempo
C= 1 #Coste de infectados, arbitrario

#Condiciones iniciales
Sh0 = 1000      # Initial susceptible humans
Ih0 = 100       # Initial infected humans
Rh0 = 50        # Initial recovered humans
Sv0 = 1000      # Initial susceptible mosquitoes
Iv0 = 100       # Initial infected mosquitoes
#The initial conditons for the adjoints, even if they should have a more important meaning for now they will be 0
lambda_1_0=0
lambda_2_0=0
lambda_3_0=0
lambda_4_0=0
lambda_5_0=0
#Faltan condiciones iniciales para el control. mas investigacion seria sugerida, basado en que es la efectividad
#que inicie en 1 deberia ser una aproximación adecuada
theta_1_0=1
theta_2_0=1

Lambda_h, omega, mu_h, beta_h, xi1, q1, delta, rho, Lambda_v, mu_v, beta_v, xi2, q2 = params
epsilon = 1
#It is noted epsilon doesn´t appear and a capital bitting rate of 1 would be weird. Data on epsilon is not
#readily available on the paper yet on the sensitivility analisis it is. Intially we will leave it as it is
#theta_1 = 1
#theta_2 = 1
#Other missing parameters, the recovery rate due to the drug, 1 is fine for tests-era en control-
C= 1 #Coste de infectados, arbitrario
d1=1 #Coste de drogas humanas, arbitrario
d2=1 #Coste de insecticidas, arbitrario
    

#Derivamos las variables adjuntas
#La primera parte deberia ser similar. El control u no es necesario y el tiempo no creo pero por si acaso se deja.
def Sistema(t, y):
    Sh, Ih, Rh, Sv, Iv,lambda_1,lambda_2,lambda_3,lambda_4,lambda_5 = y
   
    Nh = Sh + Ih + Rh
    Nv = Sv + Iv
    #Variables adjuntas
    # In the first the Iv therm is coppied twice, I think it should be Sh
    Dlambda_1_Dt = - ((Sh * beta_h * epsilon * Iv * (Ih - Rh)(lambda_2 - lambda_1)) / (Nh*Nh)) + lambda_1 * mu_h + (Sv * beta_v * epsilon * Ih(lambda_5 - lambda_4) / (Nh * Nh))
    Dlambda_2_Dt = - C - xi1 * theta_1 * (1-q1) * (lambda_3 - lambda_2) + lambda_2 * mu_h - delta * (lambda_3 - lambda_2) + Sh*beta_h*epsilon*Iv/(Nh^2) - (Sv*beta_v*epsilon*(Sh+Rh)*(lambda_5-lambda_4)/(Nh^2))#sospecho que el último termino es realmente positivo o el penultmo negativo
    Dlambda_3_Dt =  - omega*lambda_1 + mu_h*lambda_3 + (Sh*beta_h*epsilon*Iv*(lambda_2-lambda_1)/(Nh^2)) + (Sv*beta_v*epsilon*Ih*(lambda_5-lambda_4)/(Nh^2))
    Dlambda_4_Dt = - (lambda_4*beta_v*epsilon*Ih/Nh) + lambda_4*xi2*theta_2*(1-q2) + lambda_4*mu_v
    Dlambda_5_Dt = - C - (Sh*beta_h*epsilon(lambda_2-lambda_1)/Nh) + lambda_5*xi2*theta_2*(1-q2)+lambda_5 * mu_v
    #Sistema base
     # Ecuaciones del sistema incluyendo el término de control 'u'
    dSh_dt = Lambda_h + omega * Rh - beta_h * Iv / Nh * Sh - mu_h * Sh
    dIh_dt = beta_h * Iv / Nh * Sh - (xi1 * (1 - q1) + u) * Ih - (delta + rho + mu_h) * Ih
    dRh_dt = (xi1 * (1 - q1) + u) * Ih + delta * Ih - (omega + mu_h) * Rh
    dSv_dt = Lambda_v - beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Sv - mu_v * Sv
    dIv_dt = beta_v * Ih / Nh * Sv - (xi2 * (1 - q2) + u) * Iv - mu_v * Iv
    #control optimo segun el principio de Pontryagin
    Dtheta_1_Dt=xi1*(1-q1)*Ih*(lambda_2-lambda_3)/d1
    Dtheta_2_Dt=xi2*(1-q2)*(lambda_4*Sv-lambda_5*Iv)/d2
    
    
    return [Dlambda_1_Dt,Dlambda_2_Dt,Dlambda_3_Dt,Dlambda_4_Dt,Dlambda_5_Dt,dSh_dt,dIh_dt,dRh_dt,dSv_dt,dIh_dt,Dtheta_1_Dt,Dtheta_2_Dt]
    
    #Now to try to solve this the code from before ought to work
 # Time vector
t = np.linspace(0, 200, 400)

# Solve ODE
sol = solve_ivp(Sistema, [t.min(), t.max()], [Sh0, Ih0, Rh0, Sv0, Iv0,lambda_1_0,lambda_2_0,lambda_3_0,lambda_4_0,lambda_5_0], t_eval=t)
